In [29]:
import os

In [30]:
%pwd

'd:\\sign_language_conversion'

In [31]:
os.chdir("../")

In [32]:
%pwd

'd:\\'

In [33]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Rahulagowda004/sign_language_conversion.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Rahulagowda004"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d15121a9b37945a700eae5385a0ae54fa810a813"

In [34]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str

In [35]:
from sign_language_conversion.constants import *
from sign_language_conversion.utils.common import read_yaml, create_directories, save_json

In [36]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("model/model.pkl"),
            training_data=Path("artifacts/prepare_dataset/root_dir/data.pickle"),
            mlflow_uri="https://dagshub.com/Rahulagowda004/sign_language_conversion.mlflow",
            all_params=self.params
        )
        return eval_config

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
import json

In [38]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    @staticmethod
    def load_model(path: Path):
        with open(path, 'rb') as file:
            model_dict = pickle.load(file)
        model = model_dict['model']
        return model

    def evaluation(self):
        with open(self.config.training_data, 'rb') as file:
            data_dict = pickle.load(file)
        
        data = np.asarray(data_dict['data'])
        labels = np.asarray(data_dict['labels'])

        x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

        self.model = self.load_model(self.config.path_of_model)
        self.y_pred = self.model.predict(x_test)
        self.score = accuracy_score(y_test, self.y_pred)
        self.save_score()

    def save_score(self):
        scores = {"accuracy": self.score * 100}
        with open('scores.json', 'w') as file:
            json.dump(scores, file)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"accuracy": self.score})

            if tracking_url_type_store != "file":
                # Register the model
                mlflow.sklearn.log_model(self.model, "SGC_CONVERTER", registered_model_name="RANDOM_FOREST")
            else:
                mlflow.sklearn.log_model(self.model, "model")

In [39]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'